# Как происходит моделирование

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Определение бизнес-требований

На этом этапе мы определяем, что именно нужно сделать и зачем это нужно сделать.  
Такая постановка вопроса помогает понять, нужно ли вообще применять методы машинного обучения в данном конкретном случае.  
После этого мы запрашиваем необходимые данные и начинаем работу. При необходимости мы сможем вернуться на этот шаг.

## Подготовка данных

Загрузим данные

In [2]:
from sklearn.datasets import load_boston
dataset = load_boston()

In [7]:
dataset.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [8]:
X = pd.DataFrame(dataset['data'], columns=dataset['feature_names'])
y = pd.Series(dataset['target'])

In [9]:
X.head()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


In [10]:
print(dataset['DESCR'][:1220])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Проверим наличие пропусков

In [11]:
X.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

Посмотрим на корреляцию признаков с целевой переменной:

In [12]:
trimmed_feature_names = []
for i in range(X.shape[1]):
    correlation = np.corrcoef(X.iloc[:, i], y)[0, 1]
    if abs(correlation) > 0.5:
        feature_name = dataset['feature_names'][i]
        print(feature_name, correlation)
        trimmed_feature_names.append(feature_name)

RM 0.6953599470715394
PTRATIO -0.5077866855375622
LSTAT -0.737662726174015


In [13]:
X_trimmed = X[trimmed_feature_names]
# X = X_trimmed

### Разбиение выборки

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (404, 13)
X_test: (102, 13)
y_train: (404,)
y_test: (102,)


## Обучение модели

 ### Линейная регрессия

In [16]:
from sklearn.linear_model import LinearRegression
predictor = LinearRegression()
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

### Градиентный бустинг

In [27]:
from sklearn.ensemble import GradientBoostingRegressor
predictor = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.01)
predictor.fit(X_train, y_train)
y_pred = predictor.predict(X_test)

## Валидация модели

In [17]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

3.4629790211101574

## Внедрение

В данном курсе мы почти не касаемся внедрения, но нужно помнить, что только после выполнения этого шага модель сможет приносить пользу.  
На этом шаге обычно мы оборачиваем модель в сервис или в контейнер.